In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output
import plotly.express as px
import pandas as pd
import sys, datetime, base64, time, psycopg2
sys.path.append('/jupyter/ext')
import stackql
from psycopg2.extras import RealDictCursor
from psycopg2 import ProgrammingError

conn = psycopg2.connect("dbname=stackql user=stackql host=localhost port=5444")

%load_ext stackql

In [ ]:
owner = widgets.Text(
    description='GitHub Owner',
    disabled=False
)

repo = widgets.Text(
    description='GitHub Repo',
    disabled=False
)

sprint_number = widgets.IntSlider(
    value=1,
    min=1,
    max=30,
    step=1,
    description='Sprint Number',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

start_date = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End Date',
    disabled=False
)

display(sprint_number)
display(start_date)
display(end_date)
display(owner)
display(repo)

In [ ]:
def print_overwrite(message):
    clear_output(wait=True)
    print(message)

def print_overwrite(message):
    clear_output(wait=True)
    print(message)

In [ ]:
# Function to create a downloadable CSV link
def create_download_link(df, title="Download CSV file", filename="data.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = f'<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    return HTML(html)

In [ ]:
def run_stackql_queries(queries, debug=False, rate_limit_per_min=0):
    start_time = time.time()
    all_results = []
    
    # Calculate sleep time in seconds, but only if rate_limit_per_min > 0
    sleep_time = 60 / rate_limit_per_min if rate_limit_per_min > 0 else 0

    with conn.cursor(cursor_factory=RealDictCursor) as cur:
        for query in queries:
            if debug:
                print_overwrite(f"Executing: {query}...")
                
            try:
                cur.execute(query)
                results = cur.fetchall()
                
                if results:  # check if the result is not empty
                    all_results.extend(results)
                    
            except Exception as e:
                if debug:
                    print(f"Error executing query: {str(e)}")
                else:                
                    continue  # No results for this query, move on to the next one
            
            if sleep_time:
                time.sleep(sleep_time)  # Delay to respect rate limit

    df = pd.DataFrame(all_results)

    number_of_rows = df.shape[0]
    elapsed_time = round(time.time() - start_time)
    
    if debug:
        print(f"Found {number_of_rows} rows in {elapsed_time} seconds")

    return df

In [ ]:
def commit_distribution(df):
    fig = px.pie(df, names='author', values='commits', title='Distribution of Commits by Author')
    fig.show()

In [ ]:
def show_contributions(df):
    fig = px.bar(df,
             x='login',
             y='contributions',
             title='Contributions by User',
             labels={'login': 'User Login', 'contributions': 'Number of Contributions'},
             text='contributions')

    fig.update_layout(xaxis_tickangle=-45)

    fig.show()